# Kian's yt Cookbook
### Kian Hayes

This notebook serves the purpose of documenting some of the techniques and Python scripts I used to create certain YT 3D renders. I'll first go through how YT makes a 3D render to set up some background and logic for certain commands that will be used in the scripts. I'll then move on to showing specific scripts, going in depth in what's happening inside of the code along with some of the results from those scripts.

## What is 3D Rendering in YT?

3D Rendering in YT is entirely object oriented. Every piece of a render is contained within a YT class object. These objects include the data itself, the volume that data is contained in, the Scene that hosts all objects, the Transfer Function, and the camera including its' lense. All of these objects must be initialized and certain commands called wihtin those objects in order for you to produce good renders. There's a nice graphic on the YT Project website demonstrating all these objects physically that I'll link in here. To put simply, all these objects interact with each other and effect the ray tracing that goes on under the hood of YT in order to produce a 3D render. 

<PHOTO>

We'll now go into what all these objects are, what they do, and how they're called in a Python script.

## Class Objects within YT's 3D Rendering and Guided Example

This section serves the purpose of explaining all the objects associate with 3D rendering and along the way mentioning some helpful tips that I've learned that make coding more efficient or certain commands that I've found create good renders. I'll go in depth on these concepts, spending time to really explain what's going on line by line so if you just want a quick example showing a full script I've made that will produce a 3D render then skip this part.

### The Data

The data isn't necessarily an object, but it's important to call it and load it inside of your script so YT knows what data you're working with. This is done simply with the following:

In [ ]:
import yt
import unyt
from yt.visualization.volume_rendering.api import Scene, create_volume_source
from yt.visualization.volume_rendering.transfer_function_helper import TransferFunctionHelper
import numpy as np

ds = yt.load('file.h5')

First, it's required that we have the many import statements at the top there since yt does not load the specific things associated with 3D Renders with just `import yt`. We'll use numpy for many things associated with converting numbers to log space later. 

The data is typically in the format HDF5. The data that we work with is mostly HDF5 so I'm not sure if YT is capable of loading other types of data. You likely won't have to worry about that with data from the Maestro and FLASH code as they output their plt files as HDF5. The argument passed into `load` can also be a path which is often how I do it since it's likely that my python script's working directory will not be where my data is stored. Within this, you can use the wildcard character to load all data in a directory following a defined pattern as in `yt.load('some/directory/with/data/plt_cnt_*')` which is useful when making time series movies.

### The Data Volume Object

There are several different volume objects that you can pass your loaded data into but the main one that I've used is the 'Sphere' object. It is initialized by doing the following

In [ ]:
ds = yt.load('file.h5')
radius = (2e3, 'km')
core = ds.sphere(ds.domain_center, radius)

This is a simple block of code but there are a lot of YT specific things going on the arguments that I would like to explain. First is defining the radius which is specifically associated with the radius of our Data Volume Object. I've made this a tuple where the zero element is the magnitude I want the radius to be and the first element the units I want that magnitude to be. This is a good variable to define at the beginning because there will be many situations where we only need the magnitude of the radius in future commands in which we can just write `radius[0]`. In other cases where we need both we just pass in the whole variable. 

Now for the `core` assignment which creates our Data Volume Object. The general call to assign our data to a volume is `ds.<volume_type>(center, radius_of_volume)` where `volume_type` can be any volume avaible within YT. It's important to note that the `center` argument must be a 3 dimensional cartesian coordinate within our data in the form of a list as in `[x, y, z]`, although we can use `ds.domain_center` which will return the center of our data without us having to do much work. 

### Field Volume Source

Now that we have all our data inside of a volume, we need to create a seperate volume source for the specific field. This is done with the following:

In [ ]:
field = ('flash', 'temp')
my_source = create_volume_source(ds, field)

Another useful tip I'd like to explain is done here. **Define the specific field you want to render at the beginning of your script**. There are A LOT of commands in YT that require you to pass in the field you're working with. It can be a pain to type the tuple with the two strings every time we need to do this so make your life easier and put it in a variable. 

With that out of the way, we then create our Field Volume Source with `yt.create_volume_source` where the arguments are first our dataset and then second our field we want to work with. 

### The Transfer Function

The Transfer Function is how YT relates your data values in the volume source to a color that is shown in the render. Working with this object is by far the most tedious thing when making 3D renders because changing one command with this function can dramatically effect how your 3D render looks. Setting up your Transfer Function correctly is a matter of seeing defining features within data or not so it takes a lot of tweaking values and patience. The transfer function is mainly editted through the TransferFunctionHelper class which we initialize with.

In [ ]:
tfh = TransferFunctionHelper(ds)

Now that the Transfer Function is initialized, we can call some commands to change it the way we want:

In [ ]:
bounds = (1e6, 5e6)
tfh.set_log(True)
tfh.grey_opacity = False
tfh.set_bounds(bounds)
tfh.build_transfer_function()
tfh.tf.map_to_colormap(np.log10(bounds[0]), np.log10(bounds[1]), colormap="twilight")

First, I like to define my bounds for easy edits. I find myself messing with the bounds a lot and as you can see there are many times where you'll need to pass in these numbers to different commands. 

`tfh.set_log(True)` simply makes our Transfer Function set in log space. I would always keep this as True because I've found changing to linear space can cause errors. Most fields are better visualized in log space anyway.

I'm not entirely sure what `tfh.grey_opacity = False` does exaclty but I've always had it in my scripts. I believe it has to do with the transparency of your render.

`tfh.set_bounds(bounds)` simply sets your bounds to the Transfer Function. **It is very important that you define this in your script because if not, YT will try to calculate bounds for you which increase the amount of time the script runs.** Just make it easy for YT and call this to begin with!

Finally `tfh.build_transfer_function()` will update the current state of our transfer function. If you do anything to the transfer function after this it will not take effect unless you call it again. 

The next step is putting a color map on our transfer function. You can also put multiple gaussians on the transfer function but I like the results of putting the whole color map on the transfer function. `tfh.tf.map_to_colormap(np.log10(bounds[0]), np.log10(bounds[1]), colormap="twilight")` sets the max and min bounds that our color map is mapped to on our transfer function. Obviously we want it to take up our entire transfer funciton so the bounds are the same as before usually. Next we define what color map. The various color maps we can use can be found online by googling "Matplotlib Colormaps". I specifically like the way twilight looks. An example of using the Gaussians is the following (I don't use this method often because I don't like the results as much so this is the extent I'll go into with this method):

In [ ]:
nlayers = 10
tfh.tf.add_layers(
    nlayers,
    w=0.01,
    mi=np.log10(bounds[0]), # Sets the min x limit
    ma=np.log10(bounds[1]), # Sets the max x limit
    col_bounds=[-15, -8], # This changes the area of the color map that will be used
    alpha=np.logspace(-1, 6, 20), # Changes the y-axis TransferFunction plot (how high the alpha value is)
    colormap='inferno', # Changes the color map
)

Now that we have everything set up with our transfer function, we can take it and apply it to our source(s):

In [ ]:
my_source.transfer_function = tfh.tf

### The Scene and Camera

The scene is the object that hosts all the other objects. It is initialized simply with

In [ ]:
sc = Scene()

We will mainly call `sc` in order to add our objects. We want to add our source to the scene so we'll say

In [ ]:
sc.add_source(my_source)

Reminder that `my_source` is the **field volume source**, not to be confused with the data volume source. If we want to add multiple sources to the scene, we simply pass in a different source with the same command such as `sc.add_source(my_source2)`.

Now we want to add our camera so we can actually see what we're looking at. The camera will take in the ray-tracing and produce our rendered picture. It can be added to the scene with

In [ ]:
sc.add_camera(ds, lens_type="perspective")
sc.camera.focus = ds.domain_center
sc.camera.resolution = 400
sc.camera.north_vector = unyt.unyt_array([0., 1., 0.], 'km')
sc.camera.position = ds.domain_center + unyt.unyt_array([0., 0., 0.80*radius[0]], 'km')
sc.camera.set_width(radius)

`sc.add_camera` has two arguments, first the dataset, and second the lens_type which can be changed to a number of different lenses but you'll likely only use perspective. There are some whacky lenses you can use but perspective is most realistic. As you can see there are numerous things we can change about our camera such as position, resolution, and its framing. 

`sc.camera.focus = ds.domain_center` will make our camera point at the center of our data, using the same 3d coordinates as before. I haven't experimented with this parameter but I'm sure you can change it to point at different areas of interest within the data. 

`sc.camera.resolution` can take in a single integer or a tuple of integers to set the resolution of the final picture. When tweaking parameter I would keep this very low (around 300-500) because 3D rendering is an expensive process and increasing it to something like 1000 can dramatically increase your run time. 

`sc.camera.northvector = unyt.unyt_array([0., 1., 0.], 'km')` will orient your camera to a specific 3d vector. In this example, we've aligned the north vector of the camera to the y axis which means when we look at the source through our camera, the y-axis will be pointing up from our source. This is not to be confused with the normal vector to our source which would be the z axis in this case. We can think of the normal vector also being the vector from the center of our source to the camera.

`sc.camera.position = ds.domain_center + unyt.unyt_array([0., 0., 0.80*radius[0]], 'km')` places our camera at a 3D coordinate. In this example we take the data domain center and then move it out from there along the z-axis. You can mess with this based on how big your source is. There are certain fields that will create bigger sources than others so if you find that the source is too small in your final render then increase how close it is. I'm not sure why we use the yt array as I've just always had that in my script. I'm not sure if you can use an np array. 

Lastly, `sc.camera.set_width(radius)` will set how wide our camera views. I'm not sure if this is the same as field of view but I just pass in the radius to this and change the camera position if I want to change how much the camera sees. I don't change this at all. 

### Rendering

We now have everything we need to call the render statement and save our render to disk which is done through the following

In [ ]:
sc.render()

We must call the scene itself and attach the method `render()` to it which will fully conduct the processing of the current state of the scene but in order to save a picture of it we must save the render with: 

In [ ]:
if yt.is_root():
    sc.save(f'/gpfs/projects/CalderGroup/KianSpace/reu2023/plots/urca/3d_render_{field[1]}',
            render=False, 
            sigma_clip=2
    )

```python
if yt.is_root()
```
Is important when we are running our script in parallel. `yt.is_root()` will return `True` when the current running process is on the root processor, that way we aren't saving our render for however many processes we have running

```python
sc.save(f'/gpfs/projects/CalderGroup/KianSpace/reu2023/plots/urca/3d_render_{field[1]}',
            render=False, 
            sigma_clip=2
    )
```

Will save our render to the specified path. It's good practice that we give our file name something unique. This will be very important when we get into making movies with our 3D renders. 

If we do not include `render=False` to `save()` the scene will render again and we don't need that so try to include this when doing 3D renders. 

Lastly `sigma_clip=2` modifies the contrast of the scene in the final image. I like setting this to 2 because it makes the scene pretty bright and the colors vibrant

### Overview and Specific Tips

This is the overarching process of every kind of 3D render with yt. Call all of these class objects, put them in the scene, align all objects to their necessary conditions and then render. With that, we can now get into some more complicated stuff but before that, here are some useful tips to keep in mind as we continue:

- Assign the field, Transfer Function bounds, and any other repeatedly used variable at the top of your scripts in a variable. 
- When figuring out bounds for your transfer function, it is easier to just pull up a seperate Jupyter Notebook and make a SlicePlot of the same field you're trying to render and figure out your Transfer Function through that. 3D rendering is a computationally expensive process while SlicePlots can be produced in a matter of seconds. By looking at the color bar ticks on the SlicePlot, you get a feel for what your bounds should be in order for your 3D render to look good. 
    - With that, you're able to plot the current state of your transfer function with `tfh.plot()` and get a nice graph to show you what color certain values will be. This is also nice when debugging and figuring out why certain 3D renders might look weird. Here's an example:

In [ ]:
field = ('boxlib', 'radial_velocity')

sc = Scene()

radius = (1.8e3, 'km')

core = ds.sphere(ds.domain_center, radius)
my_source = create_volume_source(core, field)

nlayers = 8

bounds = (0.5e5, 10e5) 

tfh = TransferFunctionHelper(ds) 
tfh.set_field(field) 
tfh.set_log(True) 
tfh.grey_opacity = False
tfh.set_bounds(bounds) 
tfh.build_transfer_function() 

tfh.tf.map_to_colormap(np.log10(bounds[0]), np.log10(bounds[1]), colormap="twilight", scale=1e-1)

my_source.transfer_function = tfh.tf

tfh.plot()